In [1]:
'''
종속변수 연도를 할당한 후 FA 분할
'''

'\n종속변수 연도를 할당한 후 FA 분할\n'

# 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np

# 전처리 함수정의

### 1. 타자
#### 1. 기본데이터 전처리
 - "ID", "선수명" 변수는 제거
 - 결측치가 하나라도 있는 행은 제거
 - 생년월일 --> 년/월/일 변수 생성
 - 생년월일 --> 연도데이터를 기반으로 나이변수 생성
 - 기존의 "생년월일" 변수는 삭제
 - 팀명데이터 --> 팀별로 원핫인코딩
 - 기존의 "팀명" 변수 삭제
    
#### 2. 타자데이터 전처리
 - 볼넷/삼진, 땅볼/뜬공 --> 변수제거 --> **논의필요**
 
#### 3. 수비데이터 전처리
 - 포지션(수비) --> 포지션별로 원핫인코딩
 - 기존의 "포지션(수비)" 변수 삭제
   
#### 4. 논의필요
 - 볼넷/삼진, 땅볼/뜬공 데이터 어떻게 처리할 것인가?

In [2]:
def hitter_preprocessing(hitter):
    
    # ID와 선수명변수 제거 및 결측행 제거
    hitter = hitter.drop(['ID', '선수명'], axis=1) # ID와 선수명은 분석에 불필요한 열이라고 판단하여 제거
    hitter = hitter.dropna().reset_index(drop=True)

    # 생년월일 전처리
    hitter['출생연도'] = hitter['생년월일'].apply(lambda x: x.split('-')[0]).astype('int')
    hitter['출생월'] = hitter['생년월일'].apply(lambda x: x.split('-')[1]).astype('int')
    hitter['출생일'] = hitter['생년월일'].apply(lambda x: x.split('-')[2]).astype('int')
    hitter['나이'] = hitter['연도'] - hitter['출생연도'] + 1
    hitter = hitter.drop('생년월일', axis=1)

    # 데뷔년도 데이터 형변환
    hitter['데뷔년도'] = hitter['데뷔년도'].astype('int')

    # 팀명데이터 원핫인코딩
    hitter = pd.get_dummies(hitter, columns=['팀명'], prefix='팀명', drop_first=False, dtype='int')

    # 타자데이터 전처리
    ### 공격데이터 형변환
    hitter['타율'] = hitter['타율'].replace('-', 0).astype('float')
    hitter['장타율'] = hitter['장타율'].replace('-', 0).astype('float')
    hitter['순수장타율'] = hitter['순수장타율'].replace('-', 0).astype('float')
    hitter = hitter.drop(['볼넷/삼진', '땅볼/뜬공'], axis=1)

    # 수비데이터 전처리
    ### 수비데이터 형변환
    hitter[['경기(수비)', '선발경기(수비)', '실책(수비)', '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '포일(수비)', '도루허용(수비)', '도루실패(수비)']] = hitter[['경기(수비)', '선발경기(수비)', '실책(수비)', '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '포일(수비)', '도루허용(수비)', '도루실패(수비)']].astype('int')

    ### 포지션 원핫인코딩
    hitter = pd.get_dummies(hitter, columns=['포지션(수비)'], prefix='포지션(수비)', drop_first=False, dtype='int')
    
    # 연봉 및 수상횟수 전처리
    hitter['연봉'] = list(map(lambda x: x.replace(',',''), hitter['연봉']))
    hitter['연봉'] = hitter['연봉'].astype('int')

    hitter['수상횟수'] = hitter['수상횟수'].astype('int')
    
    return hitter

### 2. 투수
#### 1. 기본데이터 전처리
 - "ID", "선수명" 변수는 제거
 - 결측치가 하나라도 있는 행은 제거

 - 생년월일 --> 년/월/일 변수 생성
 - 생년월일 --> 연도데이터를 기반으로 나이변수 생성
 - 기존의 "생년월일" 변수는 삭제
 - 팀명데이터 --> 팀별로 원핫인코딩
 - 기존의 "팀명" 변수 삭제

    
#### 2. 투수데이터 전처리
 - 이닝수가 0인 행은 제거 --> **논의필요**
 - 0승 0패의 승률: 0으로 처리  --> **논의필요**
 - 땅볼/뜬공, 삼진/볼넷 --> 변수제거 --> **논의필요**
 
#### 3. 수비데이터 전처리
 - 기존의 "포지션(수비)" 변수 삭제 --> 수비의 수비포지션은 only 수비
   
#### 4. 논의필요
 - 승률 --> 0승0패인경우 -로 존재
 - 평균자책점, 이닝당_출루허용률, 피안타율, 투구수/이닝, 9이닝당 삼진, 9이닝당 볼넷, 삼진/볼넷, 피장타율 --> 이닝이 0인 경우 때문에 -로존재
 - 땅볼/뜬공, 삼진/볼넷 --> 타자와 비슷한 이슈

In [3]:
def pitcher_preprocessing(pitcher):
    
    # ID와 선수명변수 제거 및 결측행 제거
    pitcher = pitcher.drop(['ID', '선수명'], axis=1) # ID와 선수명은 분석에 불필요한 열이라고 판단하여 제거
    pitcher = pitcher.dropna().reset_index(drop=True)

    # 생년월일 전처리
    pitcher['출생연도'] = pitcher['생년월일'].apply(lambda x: x.split('-')[0]).astype('int')
    pitcher['출생월'] = pitcher['생년월일'].apply(lambda x: x.split('-')[1]).astype('int')
    pitcher['출생일'] = pitcher['생년월일'].apply(lambda x: x.split('-')[2]).astype('int')
    pitcher['나이'] = pitcher['연도'] - pitcher['출생연도'] + 1
    pitcher = pitcher.drop('생년월일', axis=1)

    # 데뷔년도 데이터 형변환
    pitcher['데뷔년도'] = pitcher['데뷔년도'].astype('int')

    # 팀명데이터 원핫인코딩
    pitcher = pd.get_dummies(pitcher, columns=['팀명'], prefix='팀명', drop_first=False, dtype='int')

    # 이닝수가 1/3이상인 선수만 분석
    pitcher = pitcher[pitcher['이닝'] != 0].reset_index(drop=True)

    # 투구데이터 전처리
    pitcher['승률'] = pitcher['승률'].replace('-',0).astype('float')
    pitcher[['평균자책점', '이닝당_출루허용률', '피안타율', '투구수/이닝', '9이닝당 삼진', '9이닝당 볼넷', '피장타율']] = pitcher[['평균자책점', '이닝당_출루허용률', '피안타율', '투구수/이닝', '9이닝당 삼진', '9이닝당 볼넷', '피장타율']].astype('float')
    pitcher = pitcher.drop(['삼진/볼넷', '땅볼/뜬공'], axis=1)

    # 수비데이터 전처리
    ### 포지션(수비) 변수 삭제
    pitcher = pitcher.drop('포지션(수비)', axis=1)
    
    # 연봉 및 수상횟수 전처리
    pitcher['연봉'] = list(map(lambda x: x.replace(',',''), pitcher['연봉']))
    pitcher['연봉'] = pitcher['연봉'].astype('int')

    pitcher['수상횟수'] = pitcher['수상횟수'].astype('int')
    
    return pitcher

# 연도별로 종속변수 할당

In [4]:
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')

In [5]:
hitter_final = pd.DataFrame()

for ID in hitter['ID'].unique():

    tmp = hitter.loc[hitter['ID'] == ID].reset_index(drop=True)

    for i in range(tmp.shape[0]):
        current_year = tmp.loc[i,'연도']

        try:
            money = tmp.loc[(tmp['연도'] == (current_year+1)), '연봉'].values[0]
        except:
            money = np.nan

        tmp.loc[i, '수정연봉'] = money

    tmp = tmp.dropna().reset_index(drop=True)
    tmp.drop('연봉', axis=1, inplace=True)
    tmp = tmp.rename(columns = {'수정연봉':'연봉'})

    hitter_final = pd.concat([hitter_final, tmp]).reset_index(drop=True)

In [6]:
pitcher_final = pd.DataFrame()

for ID in pitcher['ID'].unique():

    tmp = pitcher.loc[pitcher['ID'] == ID].reset_index(drop=True)

    for i in range(tmp.shape[0]):
        current_year = tmp.loc[i,'연도']

        try:
            money = tmp.loc[(tmp['연도'] == (current_year+1)), '연봉'].values[0]
        except:
            money = np.nan

        tmp.loc[i, '수정연봉'] = money

    tmp = tmp.dropna().reset_index(drop=True)
    tmp.drop('연봉', axis=1, inplace=True)
    tmp = tmp.rename(columns = {'수정연봉':'연봉'})

    pitcher_final = pd.concat([pitcher_final, tmp]).reset_index(drop=True)

In [7]:
hitter = hitter_final.copy()
pitcher = pitcher_final.copy()

# FA 여부 분할

In [8]:
fa_people = list(hitter.loc[hitter['FA여부'] == 1, 'ID'].unique())
fa_hitter = hitter[hitter['ID'].isin(fa_people)].reset_index(drop=True)
nonfa_hitter = hitter[~hitter['ID'].isin(fa_people)].reset_index(drop=True)

In [9]:
fa_people = list(pitcher.loc[pitcher['FA여부'] == 1, 'ID'].unique())
fa_pitcher = pitcher[pitcher['ID'].isin(fa_people)].reset_index(drop=True)
nonfa_pitcher = pitcher[~pitcher['ID'].isin(fa_people)].reset_index(drop=True)

In [10]:
pitcher

,ID,선수명,데뷔년도,생년월일,팀명,연도,평균자책점,경기,승리,패배,...,병살(수비),수비율(수비),WAR,FA여부,뉴스개수(연도별정규화),뉴스개수(누적정규화),수상횟수,누적이닝,포지션 엔트로피,연봉
0,67588,윤성빈,2017.0,1999-02-26,롯데,2018,6.39,18,2,5,...,0,1.000,0.389,0,0.131754,0.131754,0.0,50.666667,0.0,"2,700"
1,77829,김광현,2007.0,1988-07-22,SK,2007,3.62,20,3,7,...,1,1.000,1.244,0,0.481259,0.481259,0.0,77.000000,0.0,"4,000"
2,77829,김광현,2007.0,1988-07-22,SK,2008,2.39,27,16,4,...,0,0.875,5.427,0,1.000000,1.481259,2.0,239.000000,0.0,"13,000"
3,77829,김광현,2007.0,1988-07-22,SK,2009,2.80,21,12,2,...,0,1.000,4.623,0,1.000000,2.481259,0.0,377.333333,0.0,"17,500"
4,77829,김광현,2007.0,1988-07-22,SK,2010,2.37,31,17,7,...,2,0.914,7.120,0,0.765470,3.246729,0.0,571.000000,0.0,"27,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,96251,박명환,1996.0,1977-06-07,두산,2006,3.46,28,7,7,...,1,0.897,3.237,0,0.283737,1.521217,0.0,679.666667,0.0,"100,000"
2346,96251,박명환,1996.0,1977-06-07,LG,2007,3.19,27,10,6,...,1,1.000,3.722,1,0.335832,1.857049,0.0,835.000000,0.0,"100,000"
2347,96251,박명환,1996.0,1977-06-07,LG,2008,8.61,5,0,3,...,0,1.000,-0.214,0,0.058981,1.916030,0.0,858.000000,0.0,"100,000"
2348,96251,박명환,1996.0,1977-06-07,LG,2009,6.19,4,0,1,...,0,1.000,0.071,0,0.066025,1.982055,0.0,874.000000,0.0,"100,000"


# 전처리

In [11]:
fa_hitter = hitter_preprocessing(fa_hitter)
nonfa_hitter = hitter_preprocessing(nonfa_hitter)
hitter = hitter_preprocessing(hitter)

fa_hitter = fa_hitter.drop('FA여부', axis=1)
nonfa_hitter = nonfa_hitter.drop('FA여부', axis=1)

fa_pitcher = pitcher_preprocessing(fa_pitcher)
nonfa_pitcher = pitcher_preprocessing(nonfa_pitcher)
pitcher = pitcher_preprocessing(pitcher)

fa_pitcher = fa_pitcher.drop('FA여부', axis=1)
nonfa_pitcher = nonfa_pitcher.drop('FA여부', axis=1)

print(hitter.shape, nonfa_hitter.shape, fa_hitter.shape)
print(pitcher.shape, nonfa_pitcher.shape, fa_pitcher.shape)

(3010, 86) (1901, 85) (1109, 85)
(2336, 81) (1772, 80) (564, 80)


# 저장

In [12]:
hitter.to_csv('선수데이터(전처리완료)/모델링용ver2/타자(모델링용).csv', index=False, encoding='utf-8-sig')
fa_hitter.to_csv('선수데이터(전처리완료)/모델링용ver2/타자_fa(모델링용).csv', index=False, encoding='utf-8-sig')
nonfa_hitter.to_csv('선수데이터(전처리완료)/모델링용ver2/타자_nonfa(모델링용).csv', index=False, encoding='utf-8-sig')


pitcher.to_csv('선수데이터(전처리완료)/모델링용ver2/투수(모델링용).csv', index=False, encoding='utf-8-sig')
fa_pitcher.to_csv('선수데이터(전처리완료)/모델링용ver2/투수_fa(모델링용).csv', index=False, encoding='utf-8-sig')
nonfa_pitcher.to_csv('선수데이터(전처리완료)/모델링용ver2/투수_nonfa(모델링용).csv', index=False, encoding='utf-8-sig')